<span style="color:darkorange; font-size:60px">Learn Jupyter with Bloomberg</span></br>
<span style="color:lightblue; font-size:30px">Interactive data visualization - bqplot</span>

In [ ]:
import bqplot as bqp
import pandas as pd
import numpy as np
import ipywidgets as widgets
import matplotlib.pyplot as mplt
import bqplot.pyplot as plt
from utils.plot_utils import set_style # Helper function for nicer looking plots.

In [ ]:
# Styling for dark background matplotlib charts
set_style()

### Why do we need to visualize our data?
There are many reasons, of course, but a few important ones which come to mind are:
1. <span style="color: orange; font-size:16px">Easier to read.</span> Think of the stock price of a company. If you want to see how the company has been doing over time, it's harder to try and read through the data in a tabular format. A line chart makes achieving this task much easier.
2. <span style="color: orange; font-size:16px">Quality check.</span> It is easier to detect outliers when looking at a chart compared to a table.
2. <span style="color: orange; font-size:16px">Telling a story.</span>  "A picture is worth a thousand words" may sound like a cliche, but it really is not. A chart well done can convey a much more compelling story than narration alone.
3. <span style="color: orange; font-size:16px">Summary statistics and other information which is derived from data can be very misleading.</span>  Point in case "Datasaurus".

[Check out the Datasaurus example](datasaurus.ipynb)

### Data visualization is, of course, not limited to charts!

Check out this [great resource](https://informationisbeautiful.net/data/) for novel ways of visualizing data. (link last retrieved 2020-10-31)

### Common chart types used in data analysis
1. Bar
2. Line
3. Scatter
4. Pie
5. Heatmap
6. Choropleth (Geo)

This is by no means an exhaustive list, there are too many to list here, but these are the most common ones! We will focus on the first three and how to use them in `bqplot`

### Static vs. dynamic Charts
We always want to use the right tool for the job. What are the differences?

- Static chart images are great for reports and other contexts where the image doesn't change frequently. Typically generated in some backend are saved to a file with come compression technique. 
- Dynamic charts do not produce an image (hand wavey), but rather draw objects in some context (like a web browser) which together compose a chart. (They can be saved to an image too, if needed). **Most importantly, they allow for _*interactions*_!** 

In [ ]:
# Some fake data
np.random.seed(0)
x_data = np.arange(1,21,1)
y_fake_line = 100 + np.random.normal(0,1,20).cumsum()
y_fake_dots = np.random.normal(1,10,20)

### Static charts of some fake data

In [ ]:
# Using matplotlib
fig, axes = mplt.subplots(1,2, figsize=(16,4))
axes[0].scatter(x_data, y_fake_dots)
axes[1].plot(x_data, y_fake_line);

### Dynamic charts of some fake data

In [ ]:
# Using bqplot
plt.figure()
plt.plot(x_data, y_fake_line)
plt.show()

<span style="color: orange; font-size:20px">Dynamic, interactive charts are excellent for applications, live dashboards and reproducible research!</span>

### The "Grammar of Graphics" in 30 seconds, a _*very*_ light introduction.
Look at the chart above. It is made of a number of components. Can you identify those components?

- There is data to begin with, illustrated by the line mark.
- Axes, both x and y.
- Scales for those axes, both linear (other contexts can have ordinal, log etc.).
- The figure object itself, which combines all these elements.
- There is some coordinate system here.

We are trying to tell a story with our data by taking all of the point above (and more) into consideration. The Grammar of Graphics is quite a dense book to read but there are many resources online with summaries.

`bqplot` is based on the Grammar of Graphics!

## Let's dive in!

In [ ]:
def generate_fake_stock_price(steps):
    return (np.arange(steps), 100 + np.random.normal(0,1,steps).cumsum())

In [ ]:
# Data
x_data, y_data = generate_fake_stock_price(100)

### BQPlot has two interfaces

### The pyplot interface

Simple interface, great for quick plots. But not very configurable!

In [ ]:
plt.figure()
plt.plot(x_data, y_data);
plt.show()

### The object model interface

Low level API. Very powerful and configurable!

In [ ]:
# Creating the bqplot chart objects
sc_x = bqp.LinearScale()
sc_y = bqp.LinearScale()

line = bqp.Lines(x=x_data, y=y_data, labels=['Fake stock price'], display_legend=True,
                 scales={'x': sc_x, 'y': sc_y})
ax_x = bqp.Axis(scale=sc_x, label='Index')
ax_y = bqp.Axis(scale=sc_y, orientation='vertical', label='y-value')

fig = bqp.Figure(marks=[line], axes=[ax_x, ax_y], title='Line Chart')
fig

### Interactivity
<span style="color: orange; font-size:18px">Everything is an object in `bqplot`</span>.

This is crucial for interactivity. Everything is drawn directly in the browser.

In [ ]:
# Checking out the line mark object
line.x

In [ ]:
line.y

#### Let's create a separate window for the bqplot chart (great JupyterLab feature)

Now we can continue exploring the different elements of the chart!

In [ ]:
# Not only retrieving data!
line.x, line.y = generate_fake_stock_price(100)

### A 'delight' factor - do you like animations?

In [ ]:
fig.animation_duration=1000 # transition animation in milliseconds

<span style="color: orange; font-size:18px">We can change anything we want on the fly, and it will update immediately!</span>

Let's try changing a few more properties.

In [ ]:
fig.title = 'A new title for my chart'

In [ ]:
line.colors = ['limegreen']

<span style="color: orange; font-size:18px">Built for the widgets ecosystem</span>

Let's build a small app to change the line data and color randomly.

In [ ]:
# Callback function for the button
def change_line(click=None):
    line.x, line.y = generate_fake_stock_price(100)
    line.colors = [bqp.CATEGORY10[np.random.randint(len(bqp.CATEGORY10))]]
    fig.title = "Line chart with color {}".format(line.colors)
    
    
btn = widgets.Button(description='Change line!', button_style='warning')
btn.on_click(change_line)
widgets.VBox([btn, fig])

In [ ]:
# If we don't like the legend
line.display_legend=False

### Some more line chart examples

##### Two lines on the y-axis

In [ ]:
x = np.arange(100)
y = np.cumsum(np.random.randn(2, 100), axis=1) #two random walks

axes_options = {'x': {'label': 'x'}, 'y': {'label': 'y'}}
fig = plt.figure(animation_duration=1000)
lines = plt.plot(x=x, y=y, colors=['darkorange', 'dodgerblue'], axes_options=axes_options)
fig

In [ ]:
# Update data of the line mark
lines.y = np.cumsum(np.random.randn(2, 100), axis=1)

##### Non-clipped lines

In [ ]:
x = np.arange(100)
y1, y2 = np.random.randn(2, 100).cumsum(axis=1)

fig = plt.figure()
plt.scales(scales={'x': bqp.LinearScale(min=10, max=90)})
line1 = plt.plot(x, y1, 'm', labels=['Clipped Line'], display_legend=True)
line2 = plt.plot(x, y2, 'y', apply_clip=False, labels=['Non clipped line'], display_legend=True)
fig

##### Hand drawing - use as an input

No grid lines, either.

In [ ]:
# Import to process datetime data
from datetime import datetime as py_dtime

# Data
price_data = pd.DataFrame(np.cumsum(np.random.randn(150, 2).dot([[0.5, 0.4], [0.4, 1.0]]), axis=0) + 100,
                          columns=['Security 1', 'Security 2'],
                          index=pd.date_range(start='01-01-2007', periods=150))

symbol = 'Security 1'
symbol2 = 'Security 2'
date_fmt = '%m-%d-%Y'
sec2_data = price_data[symbol2].values
dates = price_data.index.values
dates_actual = price_data.index.values
prices = price_data[symbol].values


# Chart
xs_hd = bqp.DateScale(min=np.datetime64(py_dtime(2007, 1, 1)))
ys_hd = bqp.LinearScale()
line_hd = bqp.Lines(x=dates_actual, y=sec2_data, scales={'x': xs_hd, 'y': ys_hd}, colors=['darkseagreen'])

handdraw = bqp.interacts.HandDraw(lines=line_hd)
xax = bqp.Axis(scale=xs_hd, label='Date', grid_lines='none')
yax = bqp.Axis(scale=ys_hd, label='Price', orientation='vertical', grid_lines='none')

bqp.Figure(marks=[line_hd], axes=[xax, yax], interaction=handdraw)

In [ ]:
line_hd.y

#### Scatter Charts

In [ ]:
# Scales for x and y axes
sc_x = bqp.LinearScale()
sc_y = bqp.LinearScale()

# Tooltip widget - new addition
ttip = bqp.Tooltip(fields=['x', 'y'], formats=['', '.2f'])

# Scatter mark
scatt = bqp.Scatter(x=x_data[:20], y=y_fake_dots, scales={'x': sc_x, 'y': sc_y},  labels=['Random scatter data'],
                    colors=bqp.CATEGORY10, display_legend=True, tooltip=ttip)

# Axes for both x and y
ax_x = bqp.Axis(scale=sc_x, label='X Axis')
ax_y = bqp.Axis(scale=sc_y, orientation='vertical', tick_format='0.0f', label="Y-Axis")

# Figure object
fig = bqp.Figure(marks=[scatt], axes=[ax_x, ax_y], title='Scatter Chart', animation_duration=1000)
fig

#### Colorful and nice, but we are looking for a trend.
Simple OLS regression.

In [ ]:
regression = np.polyfit(x_data[:20], y_fake_dots, deg=1)
polyfit = np.polyval(regression, x_data)

# Creating a line mark
ols_line = bqp.Lines(x=x_data[:20], y=polyfit, scales={'x':sc_x, 'y':sc_y}, colors=['pink'])

# Adding the regression line
fig.marks = [scatt, ols_line]

#### Okay, we have a trend line we can see. What if we could play with the data and regression parameters and see what effect it would have on the trajectory?

<span style="color: orange; font-size:18px">We can interact with our data directly with `bqplot` by clicking and draggin it. It's as easy as that.</span>

In [ ]:
scatt.enable_move = True

<span style="color: orange; font-size:18px">Try it! Now let's make a small app out of it.</span>

In [ ]:
# Some widgets
slider_scatt = widgets.IntSlider(value=2, min=1, max=5)

# Callback for linear regression degree of freedom change
def scatter_callback(click=None):
    # New regression
    regression = np.polyfit(scatt.x, scatt.y, deg=slider_scatt.value)
    polyfit = np.polyval(regression, scatt.x)
    
    with ols_line.hold_sync():
        ols_line.x = scatt.x
        ols_line.y = polyfit

# Listening for change events
scatt.observe(scatter_callback, names=['x'])
scatt.observe(scatter_callback, names=['y'])
slider_scatt.observe(scatter_callback, 'value')

widgets.VBox([slider_scatt, fig])

This is great! But can we use the regression data generated after moving the dots?

In [ ]:
ols_line.y

### Some more scatter chart examples

##### Updating data

In [ ]:
fig = plt.figure(animation_duration=1000)
x, y = np.random.rand(2, 20)
scatt = plt.scatter(x, y, colors=['yellowgreen'], axes_options=axes_options)
fig

In [ ]:
scatt.x = np.random.rand(20) * 10
scatt.y = np.random.rand(20)

#### Bar Charts

In [ ]:
# Some data
size = 100
np.random.seed(0)

x_data = range(size)
y_data = np.random.randn(size)
y_data_2 = np.random.randn(size)
y_data_3 = np.cumsum(np.random.randn(size) * 100.)

In [ ]:
x_ord = bqp.OrdinalScale()
y_sc = bqp.LinearScale()

bar = bqp.Bars(x=np.arange(1,11), y=np.arange(1,11), scales={'x': x_ord, 'y': y_sc}, colors=bqp.CATEGORY10)
ax_x = bqp.Axis(scale=x_ord)
ax_y = bqp.Axis(scale=y_sc, tick_format='0.2f', orientation='vertical')

bqp.Figure(marks=[bar], axes=[ax_x, ax_y], padding_x=0.025, padding_y=0.025, animation_duration=1000)

In [ ]:
# Changing the orientation
bar.orientation = 'horizontal'

In [ ]:
# Adding labels
bar.label_display = True
bar.label_display_format = '.1f'
bar.label_font_style = {'fill': 'white', 'font-size': '10px'}

In [ ]:
# Changing opacities
bar.opacities = [0.5, 0.2]

In [ ]:
# Updating data
bar.orientation = 'vertical'
bar.opacities = [1]
bar.y = np.arange(-5,5)

### The interaction layer

<span style="color: orange; font-size:16px">BQPlot has an interaction layer which you can overlay on any chart type. These include:</span>

1. BrushIntervalSelector
2. FastIntervalSelector
3. HandDraw (which we have seen)
4. LassoSelector
5. MultiSelector
6. PanZoom

Many others. Try them out!

[BQplot github repo with examples](https://github.com/bqplot/bqplot)

In [ ]:
# Callback for the selector
def brush_callback(change):
    if(not br_intsel.brushing):
        html_readout.value = "<h1 style='color:linen'>Selected indices: {}-{}</h1>".format(br_intsel.selected[0],br_intsel.selected[1])
        
        
# Chart
x_ord = bqp.OrdinalScale()
y_sc = bqp.LinearScale()

bar = bqp.Bars(x=x_data, y=[y_data[:20], y_data_2[:20]], 
           scales={'x': x_ord, 'y': y_sc}, padding=0.2,
           colors=bqp.CATEGORY10)

br_intsel = bqp.interacts.BrushIntervalSelector(scale=x_ord, marks=[bar])
br_intsel.observe(brush_callback, names=['selected'])
br_intsel.observe(brush_callback, names=['brushing'])

ax_x = bqp.Axis(scale=x_ord)
ax_y = bqp.Axis(scale=y_sc, orientation='vertical', tick_format='0.2f')

fig = bqp.Figure(marks=[bar], axes=[ax_x, ax_y], interaction=br_intsel)

# Some widgetry
html_readout = widgets.HTML(value="<h1 style='color:linen'>Selected indices will be displayed here</h1>")
widgets.VBox([html_readout, fig])

Grouping can take different values!

In [ ]:
# color_mode: {'auto', 'group', 'element', 'no_group'}
bar.color_mode = 'group'

In [ ]:
# color_mode: {'auto', 'group', 'element', 'no_group'}
bar.color_mode = 'element'

In [ ]:
# color_mode: {'auto', 'group', 'element', 'no_group'}
bar.color_mode = 'no_group'

<span style="color: orange; font-size:20px">Thank you!</span>